In [1]:
# Instalar biblioteca watermark para visualização de ambiente e versões de pacotes foram usados no projeto
# %pip install -q -U watermark

In [2]:
# Importar bibliotecas
from datetime import datetime

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import pearsonr, chi2_contingency

pd.set_option('future.no_silent_downcasting', True)

# Seleção de Features  

* Escolher variáveis relevantes

In [3]:
# Carregar base de dados
df=pd.read_csv("heart_disease_uci_tratado_3.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  920 non-null    float64
 6   chol      920 non-null    float64
 7   fbs       920 non-null    bool   
 8   restecg   920 non-null    object 
 9   thalch    920 non-null    float64
 10  exang     920 non-null    bool   
 11  oldpeak   920 non-null    float64
 12  slope     920 non-null    object 
 13  num       920 non-null    int64  
 14  num_reag  920 non-null    int64  
dtypes: bool(2), float64(4), int64(4), object(5)
memory usage: 95.4+ KB


In [5]:
# Separar variáveis numéricas e categóricas
variaveis_numericas = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
variaveis_categoricas = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

print("Variáveis Numéricas:", variaveis_numericas)

print("\nVariáveis Categóricas:", variaveis_categoricas)

Variáveis Numéricas: ['id', 'age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'num', 'num_reag']

Variáveis Categóricas: ['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope']


### Análise de Correlação (Numéricas): 
* Testar de correlação de Pearson para medir a relação linear entre as variáveis numéricas e a variável alvo (num_reag).

In [6]:
# Lista de variáveis numéricas 
variaveis_numericas = ["age", "trestbps", "chol", "thalch", "oldpeak"]

In [7]:
# Dicionário para armazenar resultados
resultados = []

# Loop para calcular correlação com a variável alvo
for var in variaveis_numericas:
    coef, p_valor = pearsonr(df[var], df["num_reag"])
    resultados.append({"Variável": var, 
                       "Coef_Pearson": round(coef, 3), 
                       "p_valor": round(p_valor, 4)})

In [8]:
# Converter em DataFrame
df_corr = pd.DataFrame(resultados)

print("Correlação de Pearson com a variável alvo (num_reg):")
print(df_corr)

Correlação de Pearson com a variável alvo (num_reg):
   Variável  Coef_Pearson  p_valor
0       age         0.283   0.0000
1  trestbps         0.111   0.0007
2      chol         0.084   0.0107
3    thalch        -0.382   0.0000
4   oldpeak         0.366   0.0000


**Conclusão**
* **As Mais Importantes: oldpeak, thalch e age** são as variáveis mais fortes e confiáveis para prever a doença cardíaca, com base na correlação linear.
* **Pouco Importante: trestbps** tem uma relação fraca, mas real.
* **Não Importante (Neste Teste): chol** não mostrou uma relação linear estatisticamente significante com a doença cardíaca neste conjunto de dados.

### Teste Qui-Quadrado (Categóricas)
* Aplicar o teste Qui-quadrado (χ2) para avaliar a dependência entre as variáveis categóricas e a variável alvo (num_reag).

In [9]:
# Lista de variáveis categóricas
variaveis_categoricas = ["sex", "dataset", "cp", "fbs", "restecg", "exang", "slope"]

In [10]:
# Armazenar resultados
resultados = []

for var in variaveis_categoricas:
    tabela_contingencia = pd.crosstab(df[var], df["num_reag"])
    chi2, p, dof, expected = stats.chi2_contingency(tabela_contingencia)
    
    resultados.append({
        "Variável": var,
        "Qui-Quadrado": round(chi2, 3),
        "GL": dof,
        "p_valor": round(p, 4)
    })

In [11]:
# Converter para DataFrame
df_chi2 = pd.DataFrame(resultados)

print("Teste Qui-Quadrado de Independência (variáveis categóricas vs num_reag):")
print(df_chi2)

Teste Qui-Quadrado de Independência (variáveis categóricas vs num_reag):
  Variável  Qui-Quadrado  GL  p_valor
0      sex        85.361   1   0.0000
1  dataset       157.060   3   0.0000
2       cp       268.346   3   0.0000
3      fbs        10.145   1   0.0014
4  restecg        11.428   2   0.0033
5    exang       171.167   1   0.0000
6    slope        80.064   2   0.0000


Para entender a tabela, precisamos conhecer a hipótese do teste:

* **Hipótese Nula (H₀)**: A variável em questão e a doença cardíaca são independentes. Ou seja, saber a categoria de um paciente (ex: seu sexo) não ajuda em nada a prever se ele tem a doença.
* **Hipótese Alternativa (H₁)**: As variáveis são dependentes. Existe uma associação entre elas.

A coluna mais importante para a nossa conclusão é o p_valor. A regra é a mesma da análise anterior:

* **Se p_valor < 0.05, rejeitamos a Hipótese Nula**. Isso significa que a associação observada é estatisticamente significante e a variável é relevante.
* **Se p_valor >= 0.05, não podemos rejeitar a Hipótese Nula**. A variável é considerada independente do resultado e, portanto, não é relevante.

**Análise dos Resultados**

* **Todas as suas variáveis categóricas (sex, dataset, cp, fbs, restecg, exang, slope)** têm uma associação estatisticamente significante com a presença de doença cardíaca.

## Ranking das variáveis mais relevantes para prever doença cardíaca

In [12]:
# Lista para guardar resultados
resultados = []

# --- Correlação de Pearson (numéricas) ---
for var in variaveis_numericas:
    coef, p_valor = pearsonr(df[var], df["num_reag"])
    resultados.append({
        "Variável": var,
        "Tipo": "Numérica",
        "Estatística": "Coef_Pearson",
        "Valor": round(coef, 3),
        "Forca": abs(coef),
        "p_valor": round(p_valor, 4)
    })

# --- Qui-quadrado (categóricas) ---
for var in variaveis_categoricas:
    tabela = pd.crosstab(df[var], df["num_reag"])
    chi2, p, dof, expected = chi2_contingency(tabela)
    resultados.append({
        "Variável": var,
        "Tipo": "Categórica",
        "Estatística": "Qui-Quadrado",
        "Valor": round(chi2, 3),
        "Forca": chi2,
        "p_valor": round(p, 4)
    })

In [13]:
# Converter em DataFrame e ordenar pela força
ranking = pd.DataFrame(resultados).sort_values(by="Forca", ascending=False).reset_index(drop=True)

# Exibir ranking
print("Ranking das variáveis mais relevantes para prever doença cardíaca:")
print(ranking)

Ranking das variáveis mais relevantes para prever doença cardíaca:
    Variável        Tipo   Estatística    Valor       Forca  p_valor
0         cp  Categórica  Qui-Quadrado  268.346  268.345747   0.0000
1      exang  Categórica  Qui-Quadrado  171.167  171.166826   0.0000
2    dataset  Categórica  Qui-Quadrado  157.060  157.060493   0.0000
3        sex  Categórica  Qui-Quadrado   85.361   85.361239   0.0000
4      slope  Categórica  Qui-Quadrado   80.064   80.064364   0.0000
5    restecg  Categórica  Qui-Quadrado   11.428   11.427811   0.0033
6        fbs  Categórica  Qui-Quadrado   10.145   10.144865   0.0014
7     thalch    Numérica  Coef_Pearson   -0.382    0.381864   0.0000
8    oldpeak    Numérica  Coef_Pearson    0.366    0.366138   0.0000
9        age    Numérica  Coef_Pearson    0.283    0.282700   0.0000
10  trestbps    Numérica  Coef_Pearson    0.111    0.111305   0.0007
11      chol    Numérica  Coef_Pearson    0.084    0.084140   0.0107


## Conclusão
Em resumo, os resultados sugerem que fatores relacionados à resposta ao esforço físico (thalch, oldpeak, exang, slope, cp), além de idade e sexo, têm papel preditivo mais importante na detecção de doença cardíaca.

In [14]:
# Carregando a extensão watermark
%reload_ext watermark

In [15]:
# Imprimir
# -a (autor do projeto)
# -d (data)
# -t (hora)
# -v (versões do Python e do IPython)
# -m (informações sobre a máquina (hardware e sistema operacional))
#--iversions (versões de pacotes importadas (import))
%watermark -n -a "Patrick F. R. Ribeiro" -d -t -v -m --iversions

Author: Patrick F. R. Ribeiro

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 8.30.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit

scipy     : 1.15.3
seaborn   : 0.13.2
matplotlib: 3.10.0
pandas    : 2.2.3
numpy     : 1.26.4

